In [1]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [247]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

g_key = ''
# Import API key
#from api_keys import g_key

In [248]:
# Read weather_data.csv
weather_data = pd.read_csv(r'C:\Users\Mason\Data_Bootcamp\API-Weather\WeatherPy\weather_data.csv')

# Create DataFrame based on weather_data
weatherDF = pd.DataFrame(weather_data)
weatherDF

,City Name,Country,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,Avarua,CK,75.20,73,48,11.41,-21.21,-159.78
1,Vaini,TO,73.40,88,75,12.75,-21.20,-175.20
2,Tuatapere,NZ,55.04,83,100,10.27,-46.13,167.68
3,Jiangyou,CN,70.50,49,26,2.33,31.77,104.72
4,Bambous Virieux,MU,69.80,88,75,12.75,-20.34,57.76
...,...,...,...,...,...,...,...,...
549,Paliā Kalān,IN,72.95,48,0,2.68,28.45,80.58
550,Vanino,RU,43.83,80,100,12.30,49.09,140.25
551,Stillwater,US,42.80,70,1,8.05,36.12,-97.06
552,Le Vauclin,MQ,82.40,78,34,11.41,14.55,-60.84


In [256]:
gmaps.configure(g_key)
fig = gmaps.figure()

loc1 = weather_data['Latitude']
loc2 = weather_data['Longitude']
locations = list(zip(loc1, loc2))
weights = weather_data['Humidity']
fig.add_layer(gmaps.heatmap_layer(locations, weights, max_intensity = 100))
fig

Figure(layout=FigureLayout(height='420px'))

In [250]:
hotel_df = weatherDF[weatherDF['Max Temperature'] < 80]
hotel_df = hotel_df[weatherDF['Max Temperature'] > 70]
hotel_df = hotel_df[weatherDF['Wind Speed'] < 10]
hotel_df = hotel_df[weatherDF['Cloudiness'] == 0]
hotel_df

<ipython-input-250-b9aee10e0a09>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hotel_df = hotel_df[weatherDF['Max Temperature'] > 70]
<ipython-input-250-b9aee10e0a09>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hotel_df = hotel_df[weatherDF['Wind Speed'] < 10]
<ipython-input-250-b9aee10e0a09>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hotel_df = hotel_df[weatherDF['Cloudiness'] == 0]


,City Name,Country,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
32,Saint-Philippe,RE,73.40,52,0,5.82,-21.36,55.77
80,Cayenne,GF,77.00,94,0,1.12,4.93,-52.33
155,Ambilobe,MG,71.10,72,0,4.76,-13.20,49.05
230,Port Said,EG,75.20,83,0,8.05,31.26,32.28
262,Eilat,IL,77.00,27,0,6.93,29.56,34.95
286,Chittaurgarh,IN,76.24,35,0,3.71,24.88,74.63
396,Kānt,IN,75.27,32,0,1.83,27.80,79.80
430,Tessalit,ML,77.41,17,0,7.45,20.20,1.01
510,Sahuaripa,MX,74.05,34,0,6.06,29.05,-109.23
539,Mmathubudukwane,BW,72.12,48,0,6.02,-24.60,26.43


In [251]:
hotel_name = []

for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    location = f"{latitude},{longitude}"

    response = requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=' + location + '&radius=5000&type=lodging&key=' + g_key).json()
    try:
        hotel_name.append(response['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [252]:
hotel_df['Hotel Name'] = hotel_name
hotel_df = hotel_df.dropna(subset = ['Hotel Name'])
hotel_df

,City Name,Country,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Hotel Name
32,Saint-Philippe,RE,73.40,52,0,5.82,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
80,Cayenne,GF,77.00,94,0,1.12,4.93,-52.33,Hôtel Le Dronmi
155,Ambilobe,MG,71.10,72,0,4.76,-13.20,49.05,Hôtel & Restaurant DIANA Ambilobe
230,Port Said,EG,75.20,83,0,8.05,31.26,32.28,Aracan Hotel
262,Eilat,IL,77.00,27,0,6.93,29.56,34.95,Club Hotel Eilat
286,Chittaurgarh,IN,76.24,35,0,3.71,24.88,74.63,Hotel Radhey Heritage
396,Kānt,IN,75.27,32,0,1.83,27.80,79.80,Haseeb Khan shatreng
510,Sahuaripa,MX,74.05,34,0,6.06,29.05,-109.23,Estatua de Sahuaripa
539,Mmathubudukwane,BW,72.12,48,0,6.02,-24.60,26.43,Gill Sleepover Guesthouse
549,Paliā Kalān,IN,72.95,48,0,2.68,28.45,80.58,Hotel Sleep Inn


In [253]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
coordinates = hotel_df[["Latitude", "Longitude"]]

In [254]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))